## 1️⃣ Einleitung

Diese Dokumentation beschreibt die Entwicklung einer datengetriebenen Pipeline zur Analyse der Stimmungslage (Sentiment) von Kryptowährungen basierend auf Reddit-Daten. Ziel ist es, durch automatisierte Datenerfassung, Verarbeitung und Analyse wertvolle Einblicke in Markttrends zu gewinnen. Die Implementierung umfasst die wöchentliche Erfassung von Reddit-Posts und -Kommentaren, deren Bereinigung und anschließende Sentiment-Analyse. Die Ergebnisse werden durch kontinuierliche Integrationsprozesse mittels Jenkins automatisiert verarbeitet und in einem interaktiven Dashboard mit Streamlit visualisiert.

### Relevanz

Kryptowährungen unterliegen starken Kursschwankungen, die oft durch öffentliche Meinungen und Diskussionen in sozialen Netzwerken beeinflusst werden. Eine systematische Analyse dieser Stimmungen kann helfen:
- Markttrends frühzeitig zu erkennen,
- Investitionsentscheidungen zu unterstützen,
- Risiken besser zu bewerten.

### Zielsetzung

- Automatisierte Erfassung und Speicherung von relevanten Reddit-Diskussionen über Kryptowährungen.
- Bereinigung und Vorverarbeitung der gesammelten Daten, um eine hohe Datenqualität zu gewährleisten.
- Anwendung von Sentiment-Analyseverfahren zur Kategorisierung und Quantifizierung von Stimmungen.
- Automatisierte Bereitstellung der Ergebnisse in einem Dashboard zur kontinuierlichen Überwachung.

## 2️⃣ Einrichtung des Git-Repositories

### Vorgehensweise:

1. **Erstellung eines öffentlichen Repositories**

In [ ]:
git init
git remote add origin <URL>
git add .
git commit -m "Initial commit"
git push -u origin main


2. **Erzeugung eines Personal Access Tokens für Jenkins**
   - Navigiere zu [GitHub Personal Access Tokens](https://github.com/settings/tokens)
   - Erstelle ein Token mit den erforderlichen Berechtigungen für **Repo** und **Workflows**
   - Speichere das Token sicher und hinterlege es, du brauchst es später um Authentifizierungsprozesse für CI/CD zu ermöglichen.

## 3️⃣ Einrichtung der Reddit API

### Vorgehensweise:

1. **Registrierung einer Anwendung auf Reddit**
   - Besuche die [Reddit Developer Console](https://www.reddit.com/prefs/apps)
   - Erstelle eine neue Anwendung und wähle den Typ **script**
   - Trage eine beliebige **App-Name**, **Beschreibung** und eine **Redirect URL** (z. B. `http://localhost:8080`) ein
   - Notiere dir die generierte **Client ID** und **Client Secret**

2. **Umgebungsvariablen setzen**
   - Erstelle eine `.env` Datei und speichere die Anmeldeinformationen:

In [ ]:
CLIENT_ID='deine_client_id'
CLIENT_SECRET='dein_client_secret'
USER_AGENT='dein_user_agent'

## 4️⃣ Einrichtung des Jupyter Notebooks

Um die Datenverarbeitung und Analyse interaktiv durchzuführen, wird ein **Jupyter Notebook** verwendet.

### Installation der benötigten Python-Bibliotheken

Zunächst müssen alle relevanten Pakete installiert werden:

In [ ]:
pip install praw python-dotenv psaw transformers torch jupyter

### Laden und Initialisieren der Reddit API

Das folgende Skript lädt die API-Zugangsdaten aus der `.env` Datei und stellt eine Verbindung zur Reddit API her:

In [3]:
import praw
import pandas as pd
from datetime import datetime, timedelta
import os
import psaw as ps
from dotenv import load_dotenv
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
import torch

In [4]:
# Lade die .env-Datei
dotenv_loaded = load_dotenv("zugang_reddit.env")  # Falls die Datei anders heißt, anpassen
print(f".env geladen? {dotenv_loaded}")

.env geladen? True


In [5]:
# Verbindung zur Reddit API
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    user_agent=os.getenv("USER_AGENT")
)

print("Reddit API erfolgreich verbunden!")

Reddit API erfolgreich verbunden!


### Testabfrage von Reddit-Posts

Um die Verbindung zu überprüfen, werden die fünf heißesten Posts aus dem Subreddit `CryptoCurrency` abgerufen:

In [6]:
try:
    subreddit = reddit.subreddit("CryptoCurrency")
    for post in subreddit.hot(limit=5):
        print(f"Title: {post.title}, Score: {post.score}, URL: {post.url}")
except Exception as e:
    print(f"Fehler beim Abrufen der Posts: {e}")

Title: Moon Week 58, Score: 8, URL: https://www.reddit.com/r/CryptoCurrency/comments/1iifsjr/moon_week_58/
Title: Daily Crypto Discussion - February 10, 2025 (GMT+0), Score: 12, URL: https://www.reddit.com/r/CryptoCurrency/comments/1ilt27n/daily_crypto_discussion_february_10_2025_gmt0/
Title: Just give it a minute, Score: 1713, URL: https://i.redd.it/2sz9fm4b17ie1.png
Title: Lost Fortune: Landfill Containing $750M in Bitcoin to Be Sealed Forever, Score: 683, URL: https://news.bitcoin.com/lost-fortune-landfill-containing-750m-in-bitcoin-to-be-sealed-forever/
Title: Odds of Kanye West Launching Token Plummet After He Says ‘Coins Prey on Fans’, Score: 104, URL: https://www.coindesk.com/markets/2025/02/08/odds-of-kanye-west-launching-token-plummet-after-he-says-coins-prey-on-fans


Dieser Codeblock stellt sicher, dass die Verbindung zur Reddit API erfolgreich funktioniert. Falls es zu Fehlern kommt, sollten die API-Zugangsdaten überprüft werden.

### Definition der Kryptowährungen und Subreddits

Um die relevante Diskussion zu Kryptowährungen zu analysieren, wird eine Liste von Kryptowährungen mit ihren Symbolen sowie eine Auswahl an Subreddits definiert.

Die folgende Liste enthält die wichtigsten Kryptowährungen, die in der Analyse berücksichtigt werden:

In [7]:
cryptos = [
    ("Ethereum", "ETH"),
    ("Solana", "SOL"),
    ("Avalanche", "AVAX"),
    ("Polkadot", "DOT"),
    ("Near Protocol", "NEAR"),
    ("Polygon", "MATIC"),
    ("XRP", "XRP"),
    ("Cardano", "ADA"),
    ("Chainlink", "LINK")
]

Zur Analyse der Diskussionen werden Subreddits verwendet, die sich intensiv mit Kryptowährungen und deren Marktbewegungen beschäftigen:


In [8]:
subreddits = [
    "CryptoCurrency",  # Allgemeine Diskussionen über Kryptowährungen
    "CryptoMarkets",   # Diskussionen über den Kryptomarkt und Preisbewegungen
    "CryptoTrading",   # Fokus auf Trading-Strategien und Analysen
    "Altcoin",         # Diskussionen über Altcoins (alle Kryptowährungen außer Bitcoin)
    "DeFi",            # Decentralized Finance (DeFi) und Projekte
    "BitcoinBeginners",# Für Anfänger in der Krypto-Welt
    "cryptotechnology", # Fokus auf die zugrunde liegende Blockchain-Technologie
    "cryptocurrencies", # Allgemeine Diskussionen über Kryptowährungen
    "Satoshistreetsbets", # Krypto-Wetten und Spekulationen
    "Binance"        # Diskussionen über die Binance-Plattform  
]

Diese Listen werden später verwendet, um relevante Beiträge und Kommentare aus diesen Subreddits zu extrahieren.


### Scraping von Posts und Kommentaren

Die folgende Funktion ermöglicht es, gezielt Reddit-Posts und deren Kommentare für bestimmte Kryptowährungen zu extrahieren. Dabei werden sowohl der vollständige Name als auch das Kürzel (case-insesitive) der Kryptowährung als Suchbegriffe genutzt.

In [ ]:
def scrape_reddit(start_date, end_date, mode = 'initial'):
    start_timestamp = int(start_date.timestamp()) # Startdatum in Unix-Zeitstempel umwandeln
    end_timestamp = int(end_date.timestamp())     # Enddatum in Unix-Zeitstempel umwandeln

    posts = []  # Liste für die gesammelten Posts
    comments = []  # Liste für die gesammelten Kommentare
    post_ids = set()  # Menge für die Post-IDs, um Duplikate zu vermeiden

    for crypto_name, crypto_symbol in cryptos:
        for subreddit_name in subreddits:
        subreddit = reddit.subreddit(subreddit_name)
        print(f"Scuhe nach {crypto_name} in r/{subreddit_name}")

        # Suchbegriffe in Kleinbuchstaben umwandeln
        search_terms = [crypto_name.lower(), crypto_symbol.lower()]

        for search_term in search_terms:
            for post in subreddit.search(query=search_term, sort="new", limit=None):
                if start_timestamp <= post.created_utc <= end_timestamp and post.id not in post_ids:
                    post.ids.add(post.id)

                    # Titel und Text in Kleinbuchstaben umwandeln
                    post_title = post.title.lower()
                    post_selftext = post.selftext.lower()

                    posts.append({
                            'crypto': crypto_name,
                            'search_term': search_term,
                            'subreddit': subreddit_name,
                            'post_id': post.id,
                            'title': post_title,
                            'author': str(post.author),
                            'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                            'score': post.score,
                            'num_comments': post.num_comments,
                            'selftext': post_selftext
                        })

                    print(f"Post gefunden: {post_title} (Suchbegriff: {search_term})")

                    # 🔹 Kommentare sammeln
                    post.comments.replace_more(limit=0)
                    for comment in post.comments.list():
                        comments.append({
                            'post_id': post.id,
                            'comment_id': comment.id,
                            'author': str(comment.author),
                            'created_utc': datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                            'score': comment.score,
                            'body': (comment.body or "").lower()  # 🔹 Case-Insensitive Kommentartext
                        })
    # 🔹 Daten in DataFrames umwandeln
    df_posts = pd.DataFrame(posts)
    df_comments = pd.DataFrame(comments)

    print(f"Scrape abgeschlossen: {len(df_posts)} Posts und {len(df_comments)} Kommentare gefunden.")
    return df_posts, df_comments

### Durchführung des Scrapes

Ein Beispielaufruf für die Funktion über die letzten drei Monate:

In [ ]:
# Einmaliger Scrape für die letzten 3 Monate 
start_of_period = datetime(2024, 11, 1) # Startdatum für den Scrape
now = datetime.now()  # Aktuelles Datum
print('Starte den einmaligen Scrape für die letzten 3 Monate...')
df_posts_initial, df_comments_initial = scrape_reddit(start_of_period, now, mode='initial')
# Beispiel: Lokale Weiterverarbeitung
print("Daten können jetzt bereinigt werden...")

Bei dem wöchentlichen Scrape sieht der Aufruf wie folgt aus:

In [ ]:
# **Wöchentlicher Scrape**
now = int(datetime.now().timestamp())  # Aktueller Zeitstempel
last_week = now - 7 * 24 * 60 * 60  # 7 Tage zurück
print("🕵️ Starte den wöchentlichen Scrape...")
df_posts_weekly, df_comments_weekly = scrape_reddit(last_week, now, mode="weekly")

# Beispiel: Lokale Weiterverarbeitung
print("Daten können jetzt bereinigt werden...")

Ich habe zunächst den einmaligen Scrape durchlaufen lassen und anschliessend den wöchentlichen automatisiert, dazu kommen wir später.

### Datenbereinigung und Vorverarbeitung

Diese Funktion:
- Entfernt Duplikate in Posts und Kommentaren.
- Handhabt fehlende Werte, um Datenverluste zu vermeiden.
- Konvertiert und strukturiert Zeitstempel für bessere Nachvollziehbarkeit.
- Filtert Beiträge und Kommentare basierend auf Qualität (z. B. Score).
- Entfernt Kommentare von Accounts mit  übermäßig vielen Kommentaren.

In [ ]:
def clean_data(df_posts, df_comments, comment_threshold=500):# Anpassbarer Schwellenwert für Kommentare pro Nutzer
    # 1. Duplikate entfernen
    df_posts = df_posts.drop_duplicates(subset=["post_id"])
    df_comments = df_comments.drop_duplicates(subset=["comment_id"])
    
    # 2. Fehlende Werte behandeln
    df_posts['selftext'] = df_posts['selftext'].fillna('')  # Fehlende Posttexte auffüllen
    df_comments['body'] = df_comments['body'].fillna('')  # Fehlende Kommentare auffüllen
    
    # 3. Zeitstempel konvertieren
    df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'])
    df_comments['created_utc'] = pd.to_datetime(df_comments['created_utc'])

    # 4. Datum & Uhrzeit in separate Spalten aufteilen (Daten normalisieren)
    df_posts["date"] = df_posts["created_utc"].dt.date  # YYYY-MM-DD
    df_posts["time"] = df_posts["created_utc"].dt.time  # HH:MM:SS

    df_comments["date"] = df_comments["created_utc"].dt.date
    df_comments["time"] = df_comments["created_utc"].dt.time

    # 5. Original `created_utc`-Spalte entfernen
    df_posts.drop(columns=["created_utc"], inplace=True)
    df_comments.drop(columns=["created_utc"], inplace=True)

    # 6. Filterung nach Qualität (Spam oder irrelevante Daten entfernen)
    df_posts = df_posts[df_posts['score'] > 0]  # Posts mit negativem Score entfernen
    df_comments = df_comments[df_comments['score'] > 0]  # Kommentare mit negativem Score entfernen

    # 7. Entferne Nutzer (bots) mit übermäßigen Kommentaren
    comment_counts = df_comments["author"].value_counts()
    frequent_users = comment_counts[comment_counts > comment_threshold].index  # Nutzer über Grenze
    df_comments = df_comments[~df_comments["author"].isin(frequent_users)]

    print(f"Daten bereinigt: {df_comments.shape[0]} Kommentare übrig (nach Spam-Filter).")

    return df_posts, df_comments

### Anwendung der Bereinigungsfunktion

Nach der Extraktion der Daten wird die Bereinigungsfunktion auf die Posts und Kommentare angewendet:

In [ ]:
# Bereinigen der Daten
df_posts_clean, df_comments_clean = clean_data(df_posts_initial, df_comments_initial, comment_threshold=300) # Anpassbarer Schwellenwert für Kommentare pro Nutzer


# Überprüfen, wie viele Einträge übrig sind
print(f"Bereinigte Posts: {len(df_posts_clean)}")
print(f"Bereinigte Kommentare: {len(df_comments_clean)}")

Diese Schritte stellen sicher, dass nur relevante, qualitativ hochwertige Daten in der Pipeline weiterverarbeitet werden.

### Sentiment-Analyse der Kommentare

Nach der Bereinigung der Daten wird das Sentiment für die gesammelten Kommentare mithilfe eines vortrainierten Modells analysiert.

#### Verwendetes Sentiment-Analyse-Modell

Das Modell "ElKulako/crypto-bert" stammt von ElKulako und wurde speziell für die Analyse von Kryptowährungs-Diskussionen entwickelt. Es basiert auf der BERT-Architektur und klassifiziert Texte als bullish (positiv), neutral oder bearish (negativ). Das Modell wurde auf umfangreichen Finanz- und Krypto-spezifischen Daten trainiert, wodurch es sich besonders gut für die Analyse von Reddit-Kommentaren eignet, die sich mit dem Krypto-Markt befassen.

In [ ]:
# CryptoBERT-Modell laden
MODEL_NAME = "ElKulako/crypto-bert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# 🔹 Sicherstellen, dass die Spalte "body" existiert
if "body" not in df_comments_clean.columns:
    raise ValueError("Fehler: Die CSV-Datei enthält keine 'body'-Spalte mit Kommentaren!")

# 🔹 Funktion zur Sentiment-Analyse mit CryptoBERT
def analyze_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return "neutral", 0.0  # Leere Kommentare sind neutral

    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)

    scores = F.softmax(outputs.logits, dim=1)[0]
    labels = ["bearish", "neutral", "bullish"]  # CryptoBERT nutzt diese Labels
    sentiment = labels[torch.argmax(scores).item()]
    confidence = scores.max().item()

    return sentiment, confidence

# 🔹 Sentiment für alle Kommentare berechnen
df_comments_clean["sentiment"], df_comments_clean["sentiment_confidence"] = zip(*df_comments_clean["body"].map(analyze_sentiment))

# 🔹 Debug-Ausgabe: Zeigt die ersten 5 Ergebnisse zur Überprüfung
print(df_comments_clean[["body", "sentiment", "sentiment_confidence"]].head())


### Merging der Daten

Nachdem die Sentiment-Analyse durchgeführt wurde, werden die bereinigten Posts und Kommentare zusammengeführt.

In [ ]:
# Merging der Posts & Kommentare direkt nach der Bereinigung
df_merged = df_comments_clean.merge(df_posts_clean, on="post_id", how="left")

# Fehlende Werte entfernen (optional)
df_merged.dropna(inplace=True)

Dieser Schritt stellt sicher, dass alle relevanten Kommentare mit ihren zugehörigen Posts verknüpft sind sodass wir neben den normalisierten Post- und Kommentar-Tabelle eine fertige Tabelle für Analysen haben.


### Export und Speicherung der Daten

#### Zweck der Speicherung

Die exportierten Daten enthalten die gesammelten Reddit-Posts und Kommentare sowie deren Sentiment-Analyse. Durch die Speicherung in **Google Drive** wird eine einfache Automatisierung ermöglicht, sodass die Daten regelmäßig aktualisiert und für nachfolgende Analysen oder Machine-Learning-Prozesse verfügbar sind.


#### Implementierung des Exports

Den Pfad definieren

In [ ]:
# Setze den Pfad zu deinem Google Drive Ordner
DRIVE_PATH = "G:/Meine Ablage/reddit/"
POSTS_CSV = os.path.join(DRIVE_PATH, "reddit_posts.csv")
COMMENTS_CSV = os.path.join(DRIVE_PATH, "reddit_comments.csv")
MERGED_CSV = os.path.join(DRIVE_PATH, "reddit_merged.csv")

Die folgende Funktion speichert die extrahierten und verarbeiteten Daten als CSV-Dateien.

In [ ]:
def save_initial_csv(df_new, filename):
    """Speichert die erste CSV-Datei ohne Anhängen oder Duplikat-Prüfung."""
    file_path = os.path.join(DRIVE_PATH, filename)
    try:
        df_new.to_csv(file_path, index=False, sep="|", encoding="utf-8-sig", lineterminator="\n")
        print(f"✅ Datei erfolgreich gespeichert: {file_path}")
    except Exception as e:
        print(f"Fehler beim Speichern der Datei {filename}: {e}")

def export_initial_data(df_posts, df_comments, df_merged):
    """Speichert die initialen Posts, Kommentare & gemergten Daten."""
    try:
        save_initial_csv(df_posts, "reddit_posts.csv")
        save_initial_csv(df_comments, "reddit_comments.csv")
        save_initial_csv(df_merged, "reddit_merged.csv")
    except Exception as e:
        print(f"Fehler beim Export: {e}")

In [ ]:
# Export-Funktion für den ersten Scrape aufrufen
export_initial_data(df_posts_clean, df_comments_clean, df_merged)

#### Vorteile der Speicherung in Google Drive

- **Automatisierung**: Die exportierten Daten können regelmäßig durch Skripte oder Jenkins-Jobs aktualisiert werden.
- **Zugänglichkeit**: Die gespeicherten Dateien können von verschiedenen Systemen oder Nutzern für Analysen oder Machine-Learning-Modelle verwendet werden.
- **Versionskontrolle**: Historische Daten können gespeichert werden, um Entwicklungen über die Zeit hinweg zu analysieren.

Dieser Schritt stellt sicher, dass die verarbeiteten Daten langfristig verfügbar bleiben und kontinuierlich für weitere Analysen genutzt werden können.


## 5️⃣ Automatisierung mit Jenkins

### Einrichtung von Jenkins auf Ubuntu

Um den Scraping- und Analyseprozess zu automatisieren, wird Jenkins auf einem Ubuntu-Server eingerichtet.


1. **Installation von Jenkins auf Ubuntu**

   ```bash
   sudo apt update
   sudo apt install openjdk-11-jre
   wget -q -O - https://pkg.jenkins.io/debian/jenkins.io.key | sudo apt-key add -
   sudo sh -c 'echo deb http://pkg.jenkins.io/debian-stable binary/ > /etc/apt/sources.list.d/jenkins.list'
   sudo apt update
   sudo apt install jenkins
   sudo systemctl start jenkins
   sudo systemctl enable jenkins
   ```


Nach der Installation ist Jenkins unter `http://localhost:8080` erreichbar.

2. **Erstellen eines neuen Jobs in Jenkins**
   - Öffne die Jenkins Web-Oberfläche.
   - Erstelle einen neuen **Freestyle-Projekt**-Job.
   - Füge den **GitHub Personal Access Token** ein, um Zugriff auf das Repository zu erhalten.

3. **Konfiguration des Build-Schritts**
   Der folgende Shell-Befehl wird in den Build-Schritten des Jenkins-Jobs hinzugefügt, um das Notebook auszuführen:
   

In [ ]:
# Aktiviere das Python-Environment
. /var/lib/jenkins/venv/bin/activate

# Lade die .env-Datei und setze die Variablen
set -a
. /var/lib/jenkins/workspace/reddit_crypto_scraper/.env
set +a

# Debugging: Prüfe, ob CLIENT_ID gesetzt wurde
echo "CLIENT_ID = $CLIENT_ID"

# Installiere Abhängigkeiten aus requirements.txt
pip install -r /var/lib/jenkins/workspace/reddit_crypto_scraper/requirements.txt

# Führe das Notebook mit papermill aus
papermill /var/lib/jenkins/workspace/reddit_crypto_scraper/notebooks/reddit-skript.ipynb /var/lib/jenkins/workspace/reddit_crypto_scraper/notebooks/output.ipynb

### Vorteile der Jenkins-Automatisierung

- **Regelmäßige Ausführung**: Jenkins kann so konfiguriert werden, dass das Skript automatisch täglich oder wöchentlich ausgeführt wird.
- **Monitoring & Logging**: Jenkins speichert Logs jeder Ausführung und ermöglicht eine Fehleranalyse.
- **Reproduzierbarkeit**: Durch das Laden der `.env`-Datei und die Installation aller Abhängigkeiten ist jede Ausführung identisch.

Mit dieser Konfiguration ist die gesamte Sentiment-Analyse vollständig automatisiert und kann kontinuierlich aktualisiert werden.

## 6️⃣ Analyse und Dashboard mit Streamlit

### Einrichtung der Streamlit-App

Streamlit ist ein einfaches Framework zur Erstellung interaktiver Dashboards mit Python. Die folgende Anwendung visualisiert die analysierten Sentiment-Daten.

#### Installation der benötigten Pakete

In [ ]:
```bash
pip install streamlit pandas gdown matplotlib seaborn
```

#### Aufbau der Streamlit-App

Die folgende **`app.py`** Datei enthält das vollständige Dashboard für die Visualisierung der analysierten Reddit-Daten:


In [ ]:
import streamlit as st
import pandas as pd
import gdown
import os
import matplotlib.pyplot as plt
import seaborn as sns

st.set_page_config(page_title="Krypto-Sentiment Dashboard", layout="centered")

MERGED_CSV_ID = "102W-f_u58Jvx9xBAv4IaYrOY6txk-XKL"
MERGED_CSV = "reddit_merged.csv"

@st.cache_data
def download_csv(file_id, output):
    url = f"https://drive.google.com/uc?id={file_id}"
    gdown.download(url, output, quiet=False)

@st.cache_data
def load_data():
    if not os.path.exists(MERGED_CSV):
        download_csv(MERGED_CSV_ID, MERGED_CSV)
    df = pd.read_csv(MERGED_CSV, sep="|", encoding="utf-8-sig", on_bad_lines="skip")
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["sentiment_score"] = df["sentiment"].map({"positive": 1, "neutral": 0, "negative": -1})
    return df

df_merged = load_data()

st.title("📊 Krypto-Sentiment Dashboard")

if df_merged.empty:
    st.warning("⚠️ Keine Daten verfügbar. Überprüfe Google Drive oder lade neue Daten hoch.")
else:
    st.subheader("🔥 Top 10 meist erwähnte Kryptowährungen")
    crypto_counts = df_merged["crypto"].value_counts().head(10)
    st.bar_chart(crypto_counts)

    st.subheader("💡 Sentiment-Verteilung der Coins")
    sentiment_distribution = df_merged.groupby(["crypto", "sentiment"]).size().unstack(fill_value=0)
    st.bar_chart(sentiment_distribution)

    st.subheader("📈 Verhältnis Positiv vs. Negativ")
    sentiment_ratio = df_merged[df_merged["sentiment"] != "neutral"].groupby("sentiment").size()
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.pie(sentiment_ratio, labels=sentiment_ratio.index, autopct="%1.1f%%", startangle=90, colors=["green", "red"])
    ax.axis("equal")
    st.pyplot(fig)

    st.subheader("📅 Sentiment-Entwicklung über Zeit")
    crypto_options = df_merged["crypto"].unique().tolist()
    selected_crypto = st.selectbox("Wähle eine Kryptowährung:", crypto_options, index=0)
    df_filtered = df_merged[(df_merged["crypto"] == selected_crypto) & (df_merged["sentiment"] != "neutral")]
    df_time = df_filtered.groupby(["date", "sentiment"]).size().unstack(fill_value=0)
    st.line_chart(df_time)

st.write("🔄 Dashboard wird regelmäßig mit neuen Daten aktualisiert!")

### Starten der Streamlit-App
Die App kann mit folgendem Befehl gestartet werden:

In [ ]:
```bash
streamlit run app.py
```

### Möglichkeiten zur Erweiterung

Die Anwendung kann um weitere **Visualisierungen und Features** ergänzt werden:
- **Erweiterte Sentiment-Trends**: Liniendiagramme für Langzeitanalysen.
- **Korrelationen zwischen Coins**: Heatmaps zur Analyse von Korrelationen zwischen Kryptowährungen.
- **Interaktive Benutzersteuerung**: Mehr Auswahlmöglichkeiten für den Nutzer, wie Filter für bestimmte Zeiträume oder Sentiment-Typen.
- **Integration weiterer Datenquellen**: Kombination mit Twitter-Daten oder Finanzmarktdaten zur besseren Analyse der Marktentwicklung.

Durch die Nutzung von **Matplotlib, Seaborn und Streamlit** stehen zahlreiche Möglichkeiten für kreative Datenvisualisierung zur Verfügung. Entwickler können das Dashboard kontinuierlich verbessern und an neue Anforderungen anpassen.
